1\. Write a function that converts number representation (bin<->dec<->hex)

In [1]:
def conversion():
    representation = int(input ("Input reperesntation of number 2, 10 or 16: "))
    x = input ("Input number: ")
    x_dec = int(x, representation)
    x_bin = bin(x_dec)
    x_hex = hex(x_dec)
    return x_bin, x_dec, x_hex

a2, a10, a16 = conversion()
print("Binary representation: ",a2,"\nDecimal representation: ",a10,"\nHexadecimal representation",a16)

Input reperesntation of number 2, 10 or 16: 16
Input number: 0x17
Binary representation:  0b10111 
Decimal representation:  23 
Hexadecimal representation 0x17


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [2]:
def convert(word):
    bit = [x for x in word]
    exp = int( ''.join(bit[1:9]) , 2)
    mantissa = ''.join(bit[9:32])
    m = 1
    for i in range (len(mantissa)):
        if mantissa[i]=='1':
            m += pow(1/2,i+1)
    x = m*pow(2,exp-127)
    if bit[0]=='1':
        x=x*(-1)
    return x

x = convert('00000011111000000000000000000000')
print(x)
y = convert('11000000101100000000000000000000')
print(y)

1.316553672920962e-36
-5.5


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [3]:
x=1.0
u=0
while(x!=x/2):
    x=x/2
    u+=1
print ("underflow limit: ",pow(1/2,u-1))

y=1.0
o=0
while(y!=y*2):
    y=y*2
    o+=1
print ("overflow limit: ",1.0*pow(2,o-1))

underflow limit:  5e-324
overflow limit:  8.98846567431158e+307


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [4]:
a=1
e=-12

while (a!= a+1.0*pow(10,e)):
    e_prev = e
    prev = a
    
    a=a+1.0*pow(10,e)
    e=e-1
    
    e_next = e-1
    nex = a+1.0*pow(10,e-1)
    
print("1.0 * pow(10,",e_prev,") = ",prev)
print("1.0 * pow(10,",e,") = ",a)
print("1.0 * pow(10,",e_next,") = ",nex)

1.0 * pow(10, -15 ) =  1.00000000000111
1.0 * pow(10, -16 ) =  1.0000000000011111
1.0 * pow(10, -17 ) =  1.0000000000011111


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [5]:
import math
def quadratic_a(a,b,c):
    x1 = ((-1)*b - math.sqrt(pow(b,2) - 4*a*c)) / (2*a)
    x2 = ((-1)*b + math.sqrt(pow(b,2) - 4*a*c)) / (2*a)
    return x1,x2

a1,a2 = quadratic_a(0.001,1000,0.001)
print("(a): x1=",a1,"\tx2=",a2)

#doing this we still subtract two nearly equal numbers but in the calculation of the other root
def quadratic_b(a,b,c):
    x1 = (4*a*c) / ( (2*a) * ((-1)*b + math.sqrt(pow(b,2) - 4*a*c)) )
    x2 = (4*a*c) / ( (2*a) * ((-1)*b - math.sqrt(pow(b,2) - 4*a*c)) )
    return x1,x2

b1,b2 = quadratic_b(0.001,1000,0.001)
print("(b): x1=",b1,"\tx2=",b2)  


#only numbers of the same sign are added avoiding cancellation problems between b and the square root of b*b-4ac
def quadratic_c(a,b,c):
    if b>0:
        x1 = ((-1)*b - math.sqrt(pow(b,2) - 4*a*c)) / (2*a)
        x2 = c/(a*x1)
    else:
        x1 = ((-1)*b + math.sqrt(pow(b,2) - 4*a*c)) / (2*a)
        x2 = c/(a*x1)        
    return x1,x2    

c1,c2 = quadratic_c(0.001,1000,0.001)
print("(c): x1=",c1,"\tx2=",c2)

(a): x1= -999999.999999 	x2= -9.999894245993346e-07
(b): x1= -1000010.5755125057 	x2= -1.0000000000009999e-06
(c): x1= -999999.999999 	x2= -1.000000000001e-06


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [6]:
def d(F,x,delta):
    return (F(x+delta) - F(x)) / delta

f = lambda x: x*(x-1)

for i in range(2,15,2):
    delta = pow(10,-i)
    print("delta =",delta,"\t",d(f,1,delta))

delta = 0.01 	 1.010000000000001
delta = 0.0001 	 1.0000999999998899
delta = 1e-06 	 1.0000009999177333
delta = 1e-08 	 1.0000000039225287
delta = 1e-10 	 1.000000082840371
delta = 1e-12 	 1.0000889005833413
delta = 1e-14 	 0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [7]:
import math
import time

F = lambda x: math.sqrt(1-pow(x,2))

def Riemann(f,a,b,N):
    h=(b-a)/N
    s=0
    for k in range(1,N):
        s += h*f(a+(k*h))
    return s

N = 100   
print("N =",N,": ", Riemann(F,-1,1,N))
%timeit Riemann(F,-1,1,N)

N = 500000
while True:
    start_time = time.time()
    integral = Riemann(F,-1,1,N) 
    if time.time() - start_time > 1:
        break
    N=N+100  
    
print("\nN =",N,": ",integral)
%timeit Riemann(F,-1,1,N)

N = 100 :  1.5691342555492505
159 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

N = 500400 :  1.5707963220966117
1.02 s ± 107 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
